In [777]:
# !pip install geemap

In [778]:
import ee
import geemap

# ee.Authenticate()
# ee.Initialize()

### Create an interactive map

In [779]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [780]:
point = ee.Geometry.Point([-78.556023, 44.201128])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [781]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-01-17'

In [782]:
image.get('CLOUD_COVER').getInfo()

0.24

### import AAFC label dataset



In [783]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

use the [Canada AAFC Annual Crop Inventory](https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI#bands) to create label dataset for training


![](https://www.researchgate.net/profile/Meisam-Amani/publication/345082789/figure/fig1/AS:952723008651265@1604158274050/Annual-Space-based-Crop-Inventory-ACI-produced-by-Agriculture-and-Agri-Food-Canada.png)


In [784]:
# nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())

affc = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date('2020-01-01', '2020-12-31')).first().clip(image.geometry())

Map.addLayer(affc, {}, 'AAFC')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

## Prepare for consecutive class labels

In [785]:
raw_class_values = affc.get('landcover_class_values').getInfo()

class_palette = affc.get('landcover_class_palette').getInfo()

n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))

# class_values
# class_palette

In [786]:
affc = affc.remap(raw_class_values, new_class_values).select(
    ['remapped'], ['landcover']
)
affc = affc.set('landcover_class_values', new_class_values)
affc = affc.set('landcover_class_palette', class_palette)

## Make  dataset

In [787]:
# Make the training dataset.
points = affc.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 50000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)


## Split training and validation

In [788]:
points = points.randomColumn();

training_points = points.filter('random <= 0.7');
validation_points = points.filter('random > 0.7');

Map.addLayer(training_points, {}, 'training', False)
Map.addLayer(validation_points, {}, 'validation', False)

In [789]:
print(training_points.size().getInfo())

34527


In [790]:
print(training_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-77.2002903987127, 44.762894951029]}, 'id': '0', 'properties': {'landcover': 71, 'random': 0.0997209946498836}}


### Train the classifier

In [791]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': training_points, 'properties': [label], 'scale': 30}
)

validation = image.select(bands).sampleRegions(
    **{'collection': validation_points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trainedClassifier = ee.Classifier.smileRandomForest(50).train(training, label, bands)

In [792]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': -303, 'B2': -286, 'B3': 169, 'B4': 49, 'B5': 2741, 'B6': 536, 'B7': 434, 'landcover': 71}}


# Accuracy assessment
## Training dataset

In [793]:
# Get a confusion matrix and overall accuracy for the training sample.
train_accuracy = trainedClassifier.confusionMatrix();
# trainAccuracy.getInfo()

###Training - Overall Accuracy

In [794]:
train_accuracy.accuracy().getInfo()

0.9947534711151048

###Training -  Kappa

In [ ]:
train_accuracy.kappa().getInfo()

###Training - Producer's Accuracy

In [795]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.999485949280329],
 [0.9730769230769231],
 [0.9887640449438202],
 [0],
 [0.9936461388074291],
 [0.9871382636655949],
 [0],
 [0.9803921568627451],
 [0],
 [0.9961274947870122],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0.9423076923076923],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.9909706546275395],
 [1],
 [0.9934282584884995],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0.989247311827957],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.9906472128694351],
 [0.9962385814078453],
 [0.9965095986038395]]

###Training - Comsumer's Accuracy

In [796]:
train_accuracy.consumersAccuracy().getInfo()


[[0,
  0.9950528829750939,
  1,
  0.9967637540453075,
  0,
  1,
  1,
  0,
  1,
  0,
  0.9922848664688427,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0.9987937273823885,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0.9969879518072289,
  0.9925053533190579,
  0.9917786012042612]]

# Validation dataset

In [797]:
validated = validation.classify(trainedClassifier)

In [798]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [799]:
# test_accuracy.getInfo()

## Validation - Overall Accuracy


In [800]:
test_accuracy.accuracy().getInfo()

0.5391327694992617

## Validation - Kappa

In [801]:
test_accuracy.kappa().getInfo()

0.44969026106662546

## Validation - Producer's Accuracy

In [765]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.8890583301563095],
 [0],
 [0.18975332068311196],
 [0],
 [0.16598360655737704],
 [0.14678030303030304],
 [0],
 [0],
 [0],
 [0.5532815808045166],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.0546448087431694],
 [0],
 [0.11264367816091954],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.1387434554973822],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.33304721030042916],
 [0.5589788732394366],
 [0.6180327868852459]]

## Validation - Consumers’ Accuracy

In [766]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.8520277676287906,
  0,
  0.2375296912114014,
  0,
  0.19828641370869032,
  0.20367936925098554,
  0,
  0,
  0,
  0.41415742208135237,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.15873015873015872,
  0,
  0.22477064220183487,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.2774869109947644,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.41945945945945945,
  0.4877112135176651,
  0.5366548042704626]]

### Classify the image

In [767]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trainedClassifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [768]:
class_values = affc.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]


In [769]:
class_palette = affc.get('landcover_class_palette').getInfo()
print(class_palette)

['000000', '3333ff', '996666', 'cc6699', 'e1e1e1', 'ffff00', '993399', '501b50', 'cccc00', 'cc6600', 'ffcc33', '7899f6', 'ff9900', '660000', 'dae31d', 'd6cc00', 'd2db25', 'd1d52b', 'cace32', 'c3c63a', 'b9bc44', 'a7b34d', 'b9c64e', '999900', 'e9e2b1', '92a55b', '809769', 'ffff99', '98887c', '799b93', '5ea263', '52ae77', '41bf7a', 'd6ff70', '8c8cff', 'd6cc00', 'ff7f00', '315491', 'cc9933', '896e43', '996633', '8f6c3d', 'b6a472', '82654a', 'a39069', 'b85900', 'b74b15', 'ff8a8a', 'ffcccc', '6f55ca', 'ffccff', 'dc5424', 'd05a30', 'd20000', 'cc0000', 'dc3200', 'ff6666', 'c5453b', '7442bd', 'ffcccc', 'b5fb05', 'ccff05', '07f98c', '00ffcc', 'cc33cc', '8e7672', 'b1954f', '749a66', '009900', '006600', '00cc00', 'cc9900']


In [770]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [771]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

### Visualize the result

In [772]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [773]:
# Map.add_legend(builtin_legend='NLCD')
# Map

### Export the result

Export the result directly to your computer:

In [774]:
# import os

# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [775]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [776]:
# geemap.ee_export_image_to_drive(
#     landcover, description='landcover', folder='export', scale=900
# )